# 概要
このノートブックではタスク設定と提供されるデータの概観を見ていきます。  
EDAというよりはデータの具体的なフォーマットについてなど取り扱い説明の色が強めです。  
なお私は医療の専門家ではないのでところどころ間違った言葉を使っているかもしれませんがご容赦ください。

# タスク設定
本コンペはアメリカの医師試験で行われる、**模擬診察**のデータを扱ったコンペです。  
模擬診察では何らかの症例・アイデンティティ ("case") を想定した患者役の試験官に対し、受験生が診察を行い、適切なカルテ ("patient note") を記述することが求められます。  
試験の採点基準として、カルテ中にそのcaseで取りこぼしてはならない重要な情報や症状 ("feature") をどれだけ適切に記述できているかが重要になるわけですが、本コンペではこの**カルテ採点作業の自動化**に取り組みます。

具体的には、**カルテ中に記述された重要情報を見つけ出す固有表現抽出 (NER) タスク**を行うことになります。  
データ中のカルテとアノテーションの例を下に示します。FEATUREで示された部分が抽出対象です。

In [ ]:
from spacy import displacy

# this example is from the patient note of pn_num=16.
doc = {
    "text": 'HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." ...',
    "ents": [
        {"start": 5, "end": 9, "label": "FEATURE_11"},
        {"start": 10, "end": 11, "label": "FEATURE_12"},
        {"start": 26, "end": 38, "label": "FEATURE_9"},
        {"start": 56, "end": 69, "label": "FEATURE_10"},
        {"start": 70, "end": 91, "label": "FEATURE_3"},
        {"start": 96, "end": 118, "label": "FEATURE_9"},
    ],
    "title": None
}
options = {"colors":{
    "FEATURE_3": "#f0e68c",
    "FEATURE_9": "#87ceeb",
    "FEATURE_10": "#fc9ce7",
    "FEATURE_11": "#ffffe0",
    "FEATURE_12": "#90ee90",
}}
print("raw text:")
print(doc["text"])
print()
print()
print("annotated:")
displacy.render(doc, style="ent", options=options, manual=True)

例中のFEATUREはそれぞれ以下の情報を示しています。

|  FEATURE  |  説明  |
| ---- | ---- |
|  FEATURE_3  |  Intermittent-symptoms  |
|  FEATURE_9  |  heart-pounding-OR-heart-racing  |
|  FEATURE_10 |  Few-months-duration  |
|  FEATURE_11 |  17-year  |
|  FEATURE_12 |  Male  |


各featureの説明は、提供されるfeatures.csvファイルに全て記載されています。  
featureの種類数がクラス数になりますが、caseごとにどのfeatureが存在しうるかは決まっているので実装上は候補クラスを絞れます。  
与えられたpatient_noteがどのcaseのものなのかは事前に与えられます。

コンペ中で使われる用語をまとめます。
* patient_note: カルテ。
* case: 患者役の試験官が演じている症例などの設定。10種類 (case_num=0～9) あります。
* feature: 各caseにとって重要な情報や症状。心臓の痛みのような具体的な症状だったり家族歴だったりがあります。


In [ ]:
import os
import pathlib
import pandas as pd

DATASET_ROOT = pathlib.Path("../input/nbme-score-clinical-patient-notes/")

In [ ]:
# %% コンペデータ一覧
os.listdir(DATASET_ROOT)

---------------
# patient_notes.csv (カルテ)
## 実際のカルテはpatient_notes.csv内に格納されています。
フィールドは以下の通りです
* pn_num: カルテ (**p**atient **n**ote) のid
* case_num: 試験官の症例id
* pn_history: 記載内容

In [ ]:
patient_notes = pd.read_csv(DATASET_ROOT / "patient_notes.csv")
patient_notes

### 最初の一個目のpatient noteのデータを示します。

In [ ]:
target_note = patient_notes.loc[0]

print("pn_num:")
print(target_note["pn_num"])
print()

print("case_num:")
print(target_note["case_num"])
print()

print("pn_history:")
print(target_note["pn_history"])


### 二個目に入っていたpatient noteです。
case_num (症例) が上と同じだったので必然的にカルテの内容も似ています。

In [ ]:
another_target_note = patient_notes.loc[1]

print("pn_num:")
print(another_target_note["pn_num"])
print()

print("case_num:")
print(another_target_note["case_num"])
print()

print("pn_history:")
print(another_target_note["pn_history"])

-----------
# train.csv (アノテーションデータ)
## アノテーションはtrain.csv内に格納されています。
* id: アノテーションのid。pn_num (0埋め5文字) + "_" + feature_num (0埋め3文字) で構成されています。
* pn_num, case_num: アノテーションが対応するカルテidおよび症例idです。
* feature_num: アノテーションが対応するfeature種類id。要するにクラスidです。
* location: featureが記述されているスパン。文字ベースです。スパンは複数あったり、あるいは一つもない可能性があります。
* annotation: locationで示されたスパンに実際に記載されている文字列です。



In [ ]:
train_annotations = pd.read_csv(DATASET_ROOT / "train.csv")
train_annotations

例えば最初のアノテーションではpn_num=16であるnoteの文字列696～723文字目にfeature_num 0番に関する記載があることが示されます。  
なおfeature_num 0番は家族の心筋梗塞 ("Family-history-of-MI-OR-Family-history-of-myocardial-infarction") を表します。

In [ ]:
train_annotations.loc[0]

### アノテーションに書いてあるpn_numから対応するpatient_noteを辿れます。

In [ ]:
target_annotation = train_annotations.loc[0]
print("アノテーション:")
print(target_annotation)
print()

print("対応するpatient_note:")
target_note = patient_notes.set_index("pn_num", drop=False).loc[target_annotation["pn_num"]]
print(target_note)

なおpn_numが16のpatient_noteには全部で以下のアノテーションが付与されていました。  
case 0番では重要であるはずの情報が一部欠けていたのか、いくつかのfeatureはlocationなしになっています。  
また、複数回同じfeatureの内容が出現したためにlocationが複数スパンになっている行もあります。

In [ ]:
train_annotations.loc[train_annotations["pn_num"] == 16]

### アノテーションのlocationフィールドの正しさを確認しておきましょう

In [ ]:
target_annotation = train_annotations.loc[0]
print("アノテーション:")
print(target_annotation)
print()

print("対応するpatient_note:")
target_note = patient_notes.set_index("pn_num", drop=False).loc[target_annotation["pn_num"]]
print(target_note)

#### train.csv中のannotationやlocationは文字列になっているので少し前処理が必要です

In [ ]:
def parse_location(annotation_location):
    return [list(map(int, loc_str[1:-1].split(" "))) for loc_str in annotation_location[1:-1].split(", ") if len(loc_str) > 0]

target_locations = parse_location(target_annotation["location"])
print('target_annotation["location"]:', repr(target_annotation["location"]))
print("前処理後:", repr(target_locations))

↓ アノテーションのlocationで示されたスパンに実際に記述があることが確認できます。

In [ ]:
first_start, first_end = target_locations[0]
print('target_annotation["annotation"]:', repr(target_annotation["annotation"]))
print("実際の記載:", repr(target_note["pn_history"][first_start:first_end]))
print()
print("＋前後25文字:", repr(target_note["pn_history"][first_start-25:first_end+25]))

参考  
複数のスパンがある例：

In [ ]:
target_annotation = train_annotations.set_index("id", drop=False).loc["00016_003"]
print(target_annotation)
print()

print('parse_location(target_annotation["location"]):', repr(parse_location(target_annotation["location"])))

スパンがない例：

In [ ]:
target_annotation = train_annotations.set_index("id", drop=False).loc["95333_912"]
print(target_annotation)
print()

print('parse_location(target_annotation["location"]):', repr(parse_location(target_annotation["location"])))

--------
# features.csv (重要情報の概要データ)
## 各featureの情報はfeatures.csvにまとまっています。

In [ ]:
features = pd.read_csv(DATASET_ROOT / "features.csv")
features

#### 例えばfeature_num 0番のfeatureは家族の心筋梗塞に関する情報です。

In [ ]:
features.set_index("feature_num").loc[0]["feature_text"]

### featuresは全部で143種ですがcase_numごとにありうるfeatureが決まっています。
例えばcase_num 0番におけるfeatureの全候補は以下の13種類だけです。

In [ ]:
features.loc[features["case_num"] == 0]

----------------
# test.csv (予測対象の指示データ)
本番予測データ提出時にどのpatient_noteのどのfeatureの記載を識別しなければならないかが指示されています。

本コンペはCode Competitionの設定であり、ここで見えているtest.csvはフォーマットだけ合わせられたダミーデータです。  
本番submit時にtest.csvの内容が本物に差し替えられ、提出プログラムはそのデータに基づいてその場で予測を行う必要があります。

In [ ]:
test = pd.read_csv(DATASET_ROOT / "test.csv")
test

------
# sample_submission.csv (提出ファイル例)
test.csvで指定されたfeatureのスパンをそれぞれ答えます。

In [ ]:
sample_submission = pd.read_csv(DATASET_ROOT / "sample_submission.csv")
sample_submission

-----------------
# 評価指標：F値
**アノテーションはスパンですがスコアは一文字一文字に分解して計測します。**  

例えばあるfeatureの正解スパンが[10:30]の20文字で、予測したスパンが[5:15]の10文字なら正しく予測できた (重複している) 文字数は[10:15]の5文字なので  
precision = 5 / 10 = 0.5  
recall = 5 / 20 = 0.25  
で計算します。

固有表現抽出タスクにはスパンがどこから始まりどこで終わるか完全に正しく答えないと正解にならない設定のものもありますが、今回はそこまで厳しい設定ではありません。上の例では例えば[5:15]に加えて[20:30]のスパンも予測していると、本来は一つだったスパンが予測では飛び飛びになる違和感のある予測ですが、スコアとしてはトータルで予測20文字、うち正解15文字なので、この場合は  
precision = 15 / 20  
recall = 15 / 20  
で上の例より高スコアになります。

------
#### もしこのノートブックが役立ちましたらup voteしていただけると幸いです！🎁🎁🎁